In [1]:
## import packages
from util import createGrps
from main import createGrpsV2
from groupAggMain import aggregate
import pandas as pd
import numpy as np
from lenskit.algorithms import Recommender
from lenskit.algorithms import Predictor
from lenskit.algorithms.user_knn import UserUser
from lenskit import batch
import sys

In [2]:
## Get the training data and train the user-user collaborative filterring
# read the data (about applications)
# TODO: replace by Tine's filter  data
#data = pd.read_csv('../dataset/user_with_negative_ratings_full.csv', delimiter=',')
data = pd.read_csv("../dataset/clean/user_ratings_neg_1000_20_20_1.csv", delimiter=',')
data = data.drop(columns="Unnamed: 0")
# Do not remove negatives because is a better way to generate synthetic groups (i.e. distinguish between "negative" vs "don't cares")
display(data)



,UserID,JobID,Rating
0,554,196603,1
1,554,300053,1
2,554,1078274,1
3,554,146817,1
4,554,654538,1
...,...,...,...
956220,1471233,1069395,-1
956221,1471233,1064781,-1
956222,1471233,1059618,-1
956223,1471233,1059605,-1


In [3]:
## First get the groups
# parameters for group generation
size = 10
grpNum = 5
seed = 12345
pathToPickle = "../groupGenv2/user_matrix_full_pivot_01.pkl"
pathToUserMatrix = "../dataset/clean/user_ratings_neg_1000_20_20_1.csv"
sim_thrh = 0.8

# Choose which group generator version
version = 2

# generate synthetic groups of users and display them (Version 1)
if version == 1:
    #TODO: change '../dataset/users.tsv' to match Tine's data before using option 1
    groups2 = createGrps(pd.read_csv('../dataset/users.tsv', sep='\t'), size, grpNum, seed)
else:
    groups2 = createGrpsV2(size, grpNum, pathToPickle, pathToUserMatrix, sim_thrh, seed)
display(groups2)
groups2.to_csv("syntheticGroups.csv")

: 

In [ ]:
# construct dataframe in format (user, item, rating) via column addition
df_ui = data.rename(columns={"UserID": "user", "JobID": "item", "Rating":"rating"})
# check data being read properly
display(df_ui.head(10)) 

# train UserUser collaborative filterring
user_user = UserUser(10, min_nbrs=3)  # Minimum (3) and maximum (10) number of neighbors to consider
recsys = Recommender.adapt(user_user)
recsys.fit(df_ui)

,user,item,rating
0,554,196603,1
1,554,300053,1
2,554,1078274,1
3,554,146817,1
4,554,654538,1
5,554,336293,1
6,554,640492,1
7,554,271546,1
8,554,283949,1
9,554,1066757,1


In [ ]:
# pd.set_option('display.max_rows', None)
all_ratings = []

## Create a User-Item matrix of scores so we can apply one of the aggregation strategies
# iterate through groups generated
for i, row in groups2.iterrows():
    # get the array of users from the row
    synGrp = row.iloc[0]

    # get recommendations for all group members
    ratings_grp = batch.recommend(recsys, synGrp, n=None,  n_jobs=1)
    
    # all_ratings contain array of users of the group and dataframe of ratings (item, score, user, rank)
    all_ratings.append((synGrp, ratings_grp))

In [ ]:
num_pref = 5
limit = "sig"  # cap norm sig
strat = "misery" # add mult misery pleasure

#group_preferences_add_non = aggregate(all_ratings)
group_preferences_add_cap = aggregate(all_ratings, num_pref, limit="sig", strat='add')
#group_preferences_mis_norm = aggregate(all_ratings, num_pref, limit="norm", strat='misery')

#display(group_preferences_add_non)
display(group_preferences_add_cap)
#display(group_preferences_mis_norm)

,Members,Recommendation
0,"[1165352, 668454, 838185, 1056891, 578645, 950...","[771040, 26124, 1057611, 817674, 881913]"
1,"[549231, 583914, 941975, 294558, 803135, 99662...","[797357, 263988, 224907, 922573, 630951]"
2,"[410122, 277505, 692599, 783536, 1420225, 5545...","[496239, 268240, 332720, 496240, 1009096]"
3,"[192178, 850445, 439070, 1340921, 473967, 1327...","[448890, 623117, 870378, 447936, 97493]"
4,"[988920, 485759, 481324, 1294945, 669787, 1060...","[376559, 171383, 463247, 1015362, 176215]"


In [ ]:
## Save the results of group aggregation to file to be later used in the evaluation
group_preferences_add_cap.to_csv("grpAggr_full.csv")